In [2]:
from os import listdir
from os.path import join
import matplotlib.pyplot as plt
import copy
import numpy as np
import torch
from unidepth.utils import colorize, image_grid
from unidepth.models import UniDepthV1
from unidepth.utils.visualization import colorize_np
from unidepth.utils.geometric import project_points
from PIL import Image
import open3d as o3d
import cv2
import fuse_depth as fd
%load_ext autoreload
%autoreload 2

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


SyntaxError: expected ':' (fuse_depth.py, line 128)

In [2]:
import os
import random

model = UniDepthV1.from_pretrained("lpiccinelli/unidepth-v1-vitl14")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

support_paths = []

base_path = f'../data/environment1/images/image_10.png'

for filename in listdir("supports"): 
    full_path = join("supports", filename)
    support_paths.append(full_path)

H,W  = np.array(Image.open(base_path)).shape[:2]

sup_pcds = []
for i, sup_path in enumerate(support_paths):
    T, scale, torch_intrinsics1, torch_points1, torch_depth_output = fd.pairwise_pose(base_path, sup_path, model) # base, sup order
   # if scale > 1.2 or scale < 0.9:
   #     continue
    pcd1 = o3d.io.read_point_cloud("ply2/im1.ply")
    pcd2 = o3d.io.read_point_cloud("ply2/im2.ply")
    o3d.io.write_point_cloud(f"ply2/{i}.ply", pcd2)
    pcd2 = pcd2.scale(scale, center=(0, 0, 0))
    pcd2 = pcd2.transform(T)
   # o3d.io.write_point_cloud(f"ply2/{i}.ply", pcd2)
    sup_pcds.append(pcd2)

Instantiate: dinov2_vitl14
0.8897708546107137
0.9992779264379822


In [3]:
pcd_comb = copy.deepcopy(pcd1)
for pcd in sup_pcds:
    T = fd.icp(pcd, pcd1)
    dup_pcd = copy.deepcopy(pcd) 
    dup_pcd.transform(T)
    pcd_comb += dup_pcd

o3d.io.write_point_cloud("ply2/merged.ply", pcd_comb)

Transformation is:
[[ 0.99753482  0.03856078  0.05862891 -0.18653417]
 [-0.03654743  0.99871763 -0.03503397  0.24274918]
 [-0.05990466  0.03280486  0.99766491 -0.05140095]
 [ 0.          0.          0.          1.        ]]
Transformation is:
[[ 9.99999952e-01  1.57731474e-04  2.66844638e-04 -1.71703091e-03]
 [-1.57554224e-04  9.99999767e-01 -6.64134474e-04  2.00201248e-03]
 [-2.66949330e-04  6.64092400e-04  9.99999744e-01  5.89022500e-03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


True

In [1]:
depth_min = fd.create_depth_map(np.asarray(pcd_comb.points), torch_intrinsics1.squeeze().cpu().numpy(), (H,W), 'min')
depth_min = cv2.medianBlur(depth_min.astype('float32'), 5)

depth_col_avg = colorize_np(depth_min, vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_col_avg).save("outputs/min.png")

depth_median = fd.create_depth_map(np.asarray(pcd_comb.points), torch_intrinsics1.squeeze().cpu().numpy(), (H,W), 'median')
depth_median = cv2.medianBlur(depth_median.astype('float32'), 5)

depth_col_median = colorize_np(depth_median, vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_col_avg).save("outputs/median.png")

points_tensor = torch.tensor(np.asarray(pcd_comb.points), dtype=torch.float)
points_tensor = points_tensor.view(-1, 3).unsqueeze(0).cuda()
torch_depth_mean = project_points(points_tensor, torch_intrinsics1, (H,W))
depth_pred_col_mean = colorize(torch_depth_mean.squeeze().cpu().numpy(), vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col_mean).save("outputs/mean.png")

""" point_cloud_flat = torch_points1.view(1, 3, -1)  # Shape: (B, 3, HW)
point_cloud_transposed = point_cloud_flat.transpose(1, 2)  # Shape: (B, HW, 3)
torch_depth = project_points(point_cloud_transposed, torch_intrinsics1, (H,W)) """

depth_pred_col = colorize(torch_depth_output, vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(depth_pred_col).save("outputs/original.png") 

NameError: name 'fd' is not defined

In [5]:
gt = np.load(f"../data/environment1/depths/depth_0.npy")
gt_col = colorize_np(gt, vmin=0.01, vmax=10.0, cmap="magma_r")
Image.fromarray(gt_col).save("outputs/gt.png")

In [6]:
og_depth = torch_depth_output
mean_depth = torch_depth_mean.squeeze().cpu().numpy()
valid_indices = ~np.isnan(gt)

In [7]:
squared_error = (depth_min[valid_indices] - gt[valid_indices])**2
mse = np.mean(squared_error)
print("Min Reconstructed Depth MSE:", mse)

squared_error = (depth_median[valid_indices] - gt[valid_indices])**2
mse = np.mean(squared_error)
print("Median Reconstructed Depth MSE:", mse)

squared_error = (mean_depth[valid_indices] - gt[valid_indices])**2
mse = np.mean(squared_error)
print("Mean Reconstructed Depth MSE:", mse)

squared_error = (og_depth[valid_indices] - gt[valid_indices])**2
mse = np.mean(squared_error)
print("Original Depth MSE:", mse)

Min Reconstructed Depth MSE: 6.6517444
Median Reconstructed Depth MSE: 7.461802
Mean Reconstructed Depth MSE: 7.5073895
Original Depth MSE: 7.416718


In [8]:
print("Tolerance: 0.5")
print("Min Reconstructed Depth Percent:", fd.percent_within_tolerance(depth_min, gt, 0.5))
print("Median Reconstructed Depth Percent:", fd.percent_within_tolerance(depth_median, gt, 0.5))
print("Mean Reconstructed Depth Percent:",fd. percent_within_tolerance(mean_depth, gt, 0.5))
print("Original Depth Percent:", fd.percent_within_tolerance(og_depth, gt, 0.5))


print("Tolerance: 1")
print("Min Reconstructed Depth Percent:", fd.percent_within_tolerance(depth_min, gt, 1))
print("Median Reconstructed Depth Percent:", fd.percent_within_tolerance(depth_median, gt, 1))
print("Mean Reconstructed Depth Percent:", fd.percent_within_tolerance(mean_depth, gt, 1))
print("Original Depth Percent:", fd.percent_within_tolerance(og_depth, gt, 1))

print("Tolerance: 2")
print("Min Reconstructed Depth Percent:", fd.percent_within_tolerance(depth_min, gt, 2))
print("Median Reconstructed Depth Percent:", fd.percent_within_tolerance(depth_median, gt, 2))
print("Mean Reconstructed Depth Percent:", fd.percent_within_tolerance(mean_depth, gt, 2))
print("Original Depth Percent:", fd.percent_within_tolerance(og_depth, gt, 2))

Tolerance: 0.5


TypeError: percent_within_tolerance() missing 1 required positional argument: 'valid_mask'